# Implementation of REINFORCE Algorithm

## What we did in this milestone?

We implemented the Policy Gradient algorithm REINFORCE.  To test the performance and efficacy we evaluated it using simulated data. 


Just as a reminder the REIN

Next we will present the results and the challengs that we faced




## Reinforce Deep Setup

### Loss Function

$$
L=-\sum_{k,t}Q_{k,t}log(\pi(s_{k,t},a_{k,t}))
$$

where 
$$
Q_{k,t}=\sum_{i=0}\gamma^ir_i
$$

## Challenge: n-asset dimensional action space parametrization. 

Our action space is has dimension $n$ where $n$ correponds to the number of assets in our portfolio. To parametrize our policy we followed the standard solution in the univariate case; creation a Gaussian Policy

### Gaussian Policy

$$
\pi(a |s,\boldsymbol\theta):=\frac{1}{\sigma(s,\boldsymbol\theta)\sqrt{2\pi}}exp(-\frac{(a-\mu(s,\boldsymbol\theta))^2}{2\sigma(s,\boldsymbol\theta)^2})
$$

#### Linear Parametrization

$$
\theta:=[\boldsymbol{\theta_{\mu}},\boldsymbol{\theta_{\sigma}}]
$$

$$
\mu(s,\boldsymbol\theta) :=\boldsymbol{\theta_{\mu}}^T \pmb{x}(s)
$$

$$
\sigma(s,\boldsymbol\theta) :=exp(\boldsymbol{\theta_{\sigma}}^T \pmb{x}(s))
$$

#### Gradients

$$
\nabla ln\pi(a|s,\boldsymbol\theta_\mu)=\frac{1}{\sigma(s,\boldsymbol\theta)^2}(a-\mu(s,\boldsymbol\theta))\pmb{x}(s)
$$

$$
\nabla ln\pi(a|s,\boldsymbol\theta_\sigma)=\left(\frac{(a-\mu(s,\boldsymbol\theta))^2}{\sigma(s,\boldsymbol\theta)^2} -1\right)\pmb{x}(s)
$$

**The challenge with the multivariate case is that we will need to parametrize the coviariance matrix as well** While an increase of $n*n/2$ may not be that much. If we account for the interaction of asset features (specialy if we use lags) then the number of paramters grows multiplicative $n*n*features/2$

**Solution:** We decided to  use uncorrelated parametrizations i.e. we set the covariance of each individual action in the policy to 0. 

Still we calculated the mighty gradients for the multivariate case.  





##  Multivariate Case

We have a continuous multivariate case parametrized the following way:

$$
\pi(a|s,\theta)=\frac{exp\left(\frac{1}{2}(a-\mu(s,\Theta_\mu))^T\Sigma(s,\theta_\Sigma)^{-1}(a-\mu(s,\Theta_\mu))\right)}{\sqrt{(2\pi)^k|\Sigma(s,\theta_\Sigma)|}}
$$

Where

$$
\mu(s,\Theta_\mu)=\Theta f
$$

The parametrization of $\Sigma$ will be defined as

$$
\Sigma{i,j}=\sum_k^m \theta^{\sigma}_{i,j,k}f_k
$$

$f=[f_1,...,f_m]$ is the vector of $m$ features and $\Theta$ is a row matrix $\theta_i^T=[\theta_{i,1}...\theta_{i,n}]$ . Hence each of the individual means is a linear combination of the features.


Therefore to calculuate the log gradient we need to use Matrix Calculus. first, we define

$$
u=\Theta f
$$

then we can rewrite

$$
ln(\pi(a|s,\theta))=\frac{1}{2}(a-u)^T\Sigma(s,\theta_\Sigma)^{-1}(a-u) -ln(\sqrt{(2\pi)^k|\Sigma(s,\theta_\Sigma)|})
$$

By queation  [137] from http://www.math.uwaterloo.ca/~hwolkowi//matrixcookbook.pdf
$$
\nabla ln(\pi(a|s,\theta_{i,k}^\mu))=Tr[\frac{\partial g(u)}{\partial u}^T \frac{\partial u}{\partial \theta_{i,k}^\mu }]
$$

And then by equation [86] http://www.math.uwaterloo.ca/~hwolkowi//matrixcookbook.pdf
 
$$
\nabla ln(\pi(a|s,\theta_{i,k}^\mu))=Tr[(\Sigma^{-1}(a-\Theta f))^T I[i]*f_k]
$$

Where $I[i]$ is a row vector  with zeros in all rows and 1 in row $i$ This can expressed as

$$
\nabla ln(\pi(a|s,\theta_{i,k}^\mu))=\text{row}_i[\Sigma^{-1}(a-\Theta f)] * f_k
$$

For the case of  $\Sigma$ we can also follow https://stats.stackexchange.com/questions/27436/how-to-take-derivative-of-multivariate-normal-density and apply the chain rule to obtain

$$
\nabla ln(\pi(a|s,\theta_{i,j,k}^\sigma))=Tr\left[[-\frac{1}{2}(\Sigma^{-1}-\Sigma^{-1}(a-\mu(s,\Theta_\mu))(a-\mu(s,\Theta_\mu))^T \Sigma^{-1}]^T I[i=i,j=j]*f_k\right]
$$



## Evaluations

To evaluate the algorithm we  tried three different rewards functions:

* Maximum return : The Agent receives a reward on each step equals to the return of the portfolio on 1-step
* Maximum Sharpe: The Agent receives 0 as reward on each steap until the end of an episode when the agent receives the Sharpe ratio. 
* Monimum volatility : The Agent receives 0 as reward on each step until the  end of an episode when the agent receives the negative of the volatility of the portfolio. 

Important to mention that at each step the Agent observes the state that equals the previous weights in the portfolio and the last 32 log returns of each asset. 

Below we will present the case of minimum volatility as an example. The results where pretty similar for other  reward functions

In [1]:
from lib.Environment import DeepTradingEnvironment, LinearAgent
import datetime
import numpy as np
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.plotting import plot_efficient_frontier
from pypfopt.cla import CLA

out_reward_window=datetime.timedelta(days=7)
# parameters related to the transformation of data, this parameters govern an step before the algorithm
meta_parameters = {"in_bars_count": 30,
                   "out_reward_window":out_reward_window ,
                   "state_type":"in_window_out_window"}

# parameters that are related to the objective/reward function construction
objective_parameters = {"percent_commission": .001,
                        }
print("===Meta Parameters===")
print(meta_parameters)
print("===Objective Parameters===")
print(objective_parameters)


assets_simulation_details={"asset_1":{"method":"GBM","sigma":.01,"mean":.02},
                    "asset_2":{"method":"GBM","sigma":.03,"mean":.18}}

env=DeepTradingEnvironment.build_environment_from_simulated_assets(assets_simulation_details=assets_simulation_details,
                                                                     data_hash="simulation_gbm",
                                                                     meta_parameters=meta_parameters,
                                                                     objective_parameters=objective_parameters)
cov=np.array([[assets_simulation_details["asset_1"]["sigma"]**2,0],[0,assets_simulation_details["asset_2"]["sigma"]**2]])
mus=np.array([assets_simulation_details["asset_1"]["mean"],assets_simulation_details["asset_2"]["mean"]])

ef = EfficientFrontier(mus, cov)
# weights = ef.max_sharpe(risk_free_rate=0)
weights = ef.min_volatility()
print("minum volatility weights",weights)


===Meta Parameters===
{'in_bars_count': 30, 'out_reward_window': datetime.timedelta(days=7), 'state_type': 'in_window_out_window'}
===Objective Parameters===
{'percent_commission': 0.001}
minum volatility weights OrderedDict([(0, 0.8999999999996016), (1, 0.1000000000003984)])


In [2]:

x=np.array(list(weights.values())).reshape(-1,1)
p_vol=np.sqrt(np.matmul(np.matmul(x.T,cov),x))
p_sharpe=np.matmul(x.T,mus)/p_vol
linear_agent=LinearAgent(environment=env,out_reward_window_td=out_reward_window,
                         reward_function="min_vol",sample_observations=32)
linear_agent.set_plot_weights(weights=np.array(list(weights.values())),
                              benchmark_G=-p_vol.ravel()[0])

pre-sampling indices: 100%|██████████| 1373/1373 [00:15<00:00, 85.97it/s]


In [3]:
linear_agent.REINFORCE_linear_fit()

  8%|▊         | 1538/20000 [03:03<38:05,  8.08it/s]

KeyboardInterrupt: 

## Results

### The Good

* REINFORCE Converges to the expected portfolio using different reward functions.
* REINFORCE Convergence is agnostic of estimation of a covariance matrix. 
* REINFORCE Can allow us easily to add extra features to the parametrization of our function ( technical indicators, sentiment indices , etc)
* REINFORCE finds the optimal policy even when the rewards its stochastic. 

### The Bad

* The speed of convergence is pretty slow. it took around 10+ minutes for convergence. This was expected as REINFORCE algorithm is known for having a high variance. 
* The convergence was achieved by comparing the results with the expected answer. However the L2 Norm of the parametrization vectors continues to bounce even after convergence. model overfit?. 
* The convergence seem to requiry a dynamic $\alpha$ to avoid bouncing out of an optimal result.
* Naive implementation of SGD cant be paralelized. 

### The Ugly

* REINFORCE is an on-policy method.  This means that is sample-ineficient as we cant use episodes generated by old policies. How can we proove that if the market is action independent then we can reuse episodes?




## Next Steps

* Explore REINFORCE with baseline and A2C method. 
* Implement parametrization in TensorFlow to be able to use non linear and stronger optimizaers and paralelization. 
